## Data Aquisition 
### >>> Load Data from CSVs

In [1]:
import unicodecsv

In [2]:
import unicodecsv

def readCSV(filename):
    """
    Reads in the data from given file; filename
    
    INPUT : csv file
    OUTPUT: list
    """
    with open(filename, 'rb') as file:
        handle = unicodecsv.DictReader(file)
        return list(handle)

In [3]:
enrollments = readCSV('enrollments.csv') # data type: list
enrollments[0] # shows first row with col name from data file 

OrderedDict([('account_key', '448'),
             ('status', 'canceled'),
             ('join_date', '2014-11-10'),
             ('cancel_date', '2015-01-14'),
             ('days_to_cancel', '65'),
             ('is_udacity', 'True'),
             ('is_canceled', 'True')])

In [4]:
type(enrollments)

list

In [5]:
len(enrollments)

1640

In [6]:
enrollments[0]['account_key'] # shows first row x first col data

'448'

In [7]:
daily_engagement = readCSV('daily_engagement.csv')
daily_engagement[0]

OrderedDict([('acct', '0'),
             ('utc_date', '2015-01-09'),
             ('num_courses_visited', '1.0'),
             ('total_minutes_visited', '11.6793745'),
             ('lessons_completed', '0.0'),
             ('projects_completed', '0.0')])

In [8]:
len(daily_engagement) * 6

817440

In [9]:
project_submissions = readCSV('project_submissions.csv')
project_submissions[0] 

OrderedDict([('creation_date', '2015-01-14'),
             ('completion_date', '2015-01-16'),
             ('assigned_rating', 'UNGRADED'),
             ('account_key', '256'),
             ('lesson_key', '3176718735'),
             ('processing_state', 'EVALUATED')])

In [10]:
len(project_submissions) * 6

21852

In [11]:
11480+817440+21852

850772

## Data Cleaning 
### >>> Fix Data Types to Suitable Format

In [12]:
from datetime import datetime as dt

In [13]:
def convertDatetime(date):
    """
    Takes str type data, and returns in into a Python datetime object
    If there is no date given, returns None
    
    INPUT : string
    OUTPUT: datetime obj or None
    """
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')

In [14]:
def convertInt(i):
    """
    Takes a string which is either an empty string or represents an integer,
    and returns an int or None.
    
    INPUT : string
    OUTPUT: int or None
    """
    if i == '':
        return None
    else:
        return int(i)

In [15]:
# Clean up the data types in the enrollments table
# this can be run only once, if I run it again, the data type is already converted, and will cause error

for student in enrollments:
    student['cancel_date'] = convertDatetime(student['cancel_date'])  # str -> datetime obj
    student['days_to_cancel'] = convertInt(student['days_to_cancel']) # str -> int
    student['is_canceled'] = student['is_canceled'] == 'True'         # str -> bool
    student['is_udacity'] = student['is_udacity'] == 'True'
    student['join_date'] = convertDatetime(student['join_date'])

In [16]:
enrollments[0] # shows data in suitable/meaningful format

OrderedDict([('account_key', '448'),
             ('status', 'canceled'),
             ('join_date', datetime.datetime(2014, 11, 10, 0, 0)),
             ('cancel_date', datetime.datetime(2015, 1, 14, 0, 0)),
             ('days_to_cancel', 65),
             ('is_udacity', True),
             ('is_canceled', True)])

In [17]:
# Clean up the data types in the engagement table

for record in daily_engagement:
    record['lessons_completed'] = int(float(record['lessons_completed']))    # str -> float ->int
    record['num_courses_visited'] = int(float(record['num_courses_visited']))
    record['projects_completed'] = int(float(record['projects_completed']))
    record['total_minutes_visited'] = float(record['total_minutes_visited']) # str -> float
    record['utc_date'] = convertDatetime(record['utc_date'])

In [18]:
daily_engagement[0]

OrderedDict([('acct', '0'),
             ('utc_date', datetime.datetime(2015, 1, 9, 0, 0)),
             ('num_courses_visited', 1),
             ('total_minutes_visited', 11.6793745),
             ('lessons_completed', 0),
             ('projects_completed', 0)])

In [19]:
# Clean up the data types in the submissions table

for submission in project_submissions:
    submission['completion_date'] = convertDatetime(submission['completion_date'])
    submission['creation_date'] = convertDatetime(submission['creation_date'])

In [20]:
project_submissions[0]

OrderedDict([('creation_date', datetime.datetime(2015, 1, 14, 0, 0)),
             ('completion_date', datetime.datetime(2015, 1, 16, 0, 0)),
             ('assigned_rating', 'UNGRADED'),
             ('account_key', '256'),
             ('lesson_key', '3176718735'),
             ('processing_state', 'EVALUATED')])

## Questions
### – After briefly looking at the data, what interesting question can we ask?

- How long each student takes to submit their projects?
- How do students who passes their projects differ from those who don't in terms of daily engagement?
- How many do students visit courses and for how long do they visit before cancel?
- Among visited courses, how many lessons and projects were completed?
- Any relationship between the number of completed lessons and the number of completed project?
- How the daily engagement of each student changes?
- How many times students submit their projects before it passes?

## Explore Data

### >>> Closer look to the data

In [21]:
print (type(enrollments))
print (type(student))

<class 'list'>
<class 'collections.OrderedDict'>


In [22]:
print ("total number of rows:", len(enrollments))
print("total number of columns:", len(student))

total number of rows: 1640
total number of columns: 7


In [23]:
def studentCounter(data):
    """
    Returns the total number of rows in data set
    and the the number of unique account key.
    
    INPUT : list
    OUTPUT: tuple
    """
    
    unique = set() # "set" is unordered collections of unique elements

    for row in data:
        try:
            unique.add(row['account_key'])
            
        # this is just a hack to use same function for all three data file 
        # it's not good practice and the fixed version is in several cells below
        except Exception:
            unique.add(row['acct'])

    return (len(data), len(unique))

In [24]:
studentCounter(enrollments) # among 1640 student info, there are 338(1640-1302) duplicated ones

(1640, 1302)

In [25]:
studentCounter(daily_engagement) # 136240 daily records info is about 1237 unique students

(136240, 1237)

In [26]:
studentCounter(project_submissions) # total of 3642 submissions are from 743 students

(3642, 743)

### >>> Observations

(1) The number of row is bigger than the number of unique students.
- This might be due to the students enrolled and cancelled and re-enrolled again with same account info.
    
(2) __The unique number of students from daily engagement is smaller than the number of enrolled students.__
- This might be because either some students enrolled to the course but didn't start anything yet.
- Yet, still the num of unique students in enrollments and daily engagement should match!!!
- _We are going to fix it in below_

(3) About only half of enrolled students submitted projects.

(4)__ There are two different names that holds same type of info; 'account_key' and 'acct'__
- _We are going to fix it in below_


## More Data Cleaning
### >>> Merge Multiple Column Names for the Same Data to Single Name, for (4)

In [27]:
# Rename the "acct" column in the daily_engagement table to "account_key".

for record in daily_engagement:
    # creating new column called "account_key" and copy data from 'acct' col
    record['account_key'] = record['acct']
    # delete 'acct' col
    del record['acct']    

In [28]:
daily_engagement[0]['account_key'] # getting data with new colunm name works

'0'

<b>_Now, since there will a single column name for account key data the "counter" function from above doesn't have to be hacky. It can simply look for data points in "account key" columns._

In [29]:
def uniqueStudent(data):
    """
    Returns a set only holds unique account_key info
    
    INPUT : list
    OUTPUT: set
    """
    
    unique = set()

    for row in data:
        unique.add(row['account_key'])
            
    return unique

(1-11)
### >>> Find enrollment records without corresponding daily engagement data and output that enrollment, for (2)

In [30]:
uniqueEnrolledAccounts = uniqueStudent(enrollments) # only holds 'account_key' info 
uniqueActiveAccounts = uniqueStudent(daily_engagement)

In [31]:
len(uniqueEnrolledAccounts)

1302

In [32]:
len(uniqueActiveAccounts)

1237

In [33]:
# nonActiveAccount list holds 
# unique "account_key" of students who are enrolled, but doesn't have daily engagement record

uniqueNonActiveAccounts = []

for accountKey in uniqueEnrolledAccounts:
    if accountKey not in uniqueActiveAccounts:
        uniqueNonActiveAccounts.append(accountKey)

In [34]:
len(uniqueNonActiveAccounts) 

65

<b> _There are 65 enrolled students that has no activity records. Which also makes sense by doing simple math, 1302-1237 = 65.
<br> <br>
Below is first 5 non-active students account keys._

In [35]:
print(uniqueNonActiveAccounts[:5])

['1148', '739', '875', '1063', '1237']


<i>__However, just looking at 65 account keys won't tell us much. Having enrollment information of those non-active unique student like below might be more helpful.__

In [36]:
# nonActiveStudents list holds ENROLLMENT DATA(not just account_key) of students 
# who are enrolled but doesn't have daily engagement record

nonActiveStudents = []

for student in enrollments: # "student" will have all enrollment data not just accout_key
    if student['account_key'] not in uniqueActiveAccounts:
        nonActiveStudents.append(student) 

In [37]:
len(nonActiveStudents) 

71

<i>__Wait, why the length of "uniqueNonActiveAccounts(65)" and "nonActiveStudents(71)" are different?__ <br>

__uniqueNonActiveAccounts__ is composition of _unique_ enrolled accounts that are not active. Whereas, __nonActiveStudents__ is composed by all enrollement info, _including duplicates_, that are not active.

Remember? Students info for uniqueNonActiveAccounts is from "uniqueEnrolledAccounts" which we removed all the duplicates. On the other hand, the student info for uniqueNonActiveStudents is from the whole "enrollments" which still has all the duplicated account key. <br>
<br>
__So, what are those duplicated ones?__

In [38]:
accntkeyFreq = {}

# adding all nonActiveStudents to dict
for accountKey in uniqueNonActiveAccounts:
    accntkeyFreq[accountKey] = 0

# counting frequency of each accout key that occurs in nonActiveStudents
for student in nonActiveStudents:
    accntkeyFreq[student['account_key']] += 1

# these are the duplicated ones 
for key in accntkeyFreq:
    if accntkeyFreq[key] > 1:
        print ("key:", key, " freq:", accntkeyFreq[key])

key: 1129  freq: 2
key: 914  freq: 2
key: 819  freq: 2
key: 1304  freq: 2
key: 654  freq: 3


<i><b> Now that's clear, let's look at the first 5 non-active students enrollment info.

In [39]:
print(nonActiveStudents[:5]) 

[OrderedDict([('account_key', '1219'), ('status', 'canceled'), ('join_date', datetime.datetime(2014, 11, 12, 0, 0)), ('cancel_date', datetime.datetime(2014, 11, 12, 0, 0)), ('days_to_cancel', 0), ('is_udacity', False), ('is_canceled', True)]), OrderedDict([('account_key', '871'), ('status', 'canceled'), ('join_date', datetime.datetime(2014, 11, 13, 0, 0)), ('cancel_date', datetime.datetime(2014, 11, 13, 0, 0)), ('days_to_cancel', 0), ('is_udacity', False), ('is_canceled', True)]), OrderedDict([('account_key', '1218'), ('status', 'canceled'), ('join_date', datetime.datetime(2014, 11, 15, 0, 0)), ('cancel_date', datetime.datetime(2014, 11, 15, 0, 0)), ('days_to_cancel', 0), ('is_udacity', False), ('is_canceled', True)]), OrderedDict([('account_key', '654'), ('status', 'canceled'), ('join_date', datetime.datetime(2014, 12, 4, 0, 0)), ('cancel_date', datetime.datetime(2014, 12, 4, 0, 0)), ('days_to_cancel', 0), ('is_udacity', False), ('is_canceled', True)]), OrderedDict([('account_key', '6

<b><i> One thing to observe is that the value for 'days_to_cancel' column is 0 for all there 5 rows. This explains why some of enrolled student don't have daily engagement record which seems normal. <br>

<b><i> Out of 71 Non-active students record, 68 of them has 'days_to_cancel' == 0 and 3 of them has 'days_to_cancel' != 0 and these are Udacity test accounts.

In [40]:
for student in nonActiveStudents:
    if student['days_to_cancel'] != 0:
        print (student)

OrderedDict([('account_key', '1304'), ('status', 'canceled'), ('join_date', datetime.datetime(2015, 1, 10, 0, 0)), ('cancel_date', datetime.datetime(2015, 3, 10, 0, 0)), ('days_to_cancel', 59), ('is_udacity', True), ('is_canceled', True)])
OrderedDict([('account_key', '1304'), ('status', 'canceled'), ('join_date', datetime.datetime(2015, 3, 10, 0, 0)), ('cancel_date', datetime.datetime(2015, 6, 17, 0, 0)), ('days_to_cancel', 99), ('is_udacity', True), ('is_canceled', True)])
OrderedDict([('account_key', '1101'), ('status', 'current'), ('join_date', datetime.datetime(2015, 2, 25, 0, 0)), ('cancel_date', None), ('days_to_cancel', None), ('is_udacity', True), ('is_canceled', False)])


In [41]:
cnt = 0
for student in nonActiveStudents:
    if student['days_to_cancel'] == 0:
        cnt += 1

print (cnt)

68


<b><i> Maybe it's better to remove Udacity test accounts from all three datasets. 

<b><i>First, how many are there?
--> 4 from non active students and 18 from total enrolled accounts 

In [42]:
cnt = 0

for student in nonActiveStudents:
    if student['is_udacity']:
        cnt += 1

print (cnt)

4


In [43]:
cnt = 0
udacity_AccountKey = set() # only to store unique values 

for student in enrollments:
    if student['is_udacity']:
        udacity_AccountKey.add(student['account_key'])
        cnt += 1

print (cnt)

18


In [44]:
udacity_AccountKey

{'1069', '1101', '1304', '312', '448', '818'}

In [45]:
# function to get only data without Udacity test account

def remove_isUdacity(data):
    resultList = []
    for row in data:
        if row['account_key'] in udacity_AccountKey:
            pass
        else:
            resultList.append(row)
    
    return resultList

<b><i> Remove all the rows from all three datasets that are Udacity test account by using account_key 

In [46]:
enrollments_noUdacity = remove_isUdacity(enrollments)
daily_engagement_noUdacity = remove_isUdacity(daily_engagement)
project_submissions_noUdacity = remove_isUdacity(project_submissions)

In [47]:
print (len(enrollments_noUdacity)) # total: 1640, udacity account: 18, 1640 - 18 = 1622 --> seems like it works!
print (len(daily_engagement_noUdacity))
print (len(project_submissions_noUdacity))

1622
135656
3634


... start from (1-14) 